In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.multiclass import OneVsRestClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# ----- Csv files and paths used to classify music ----
TRAIN_PATH = '../input/tsma-202223-music-genre-classification/train/Train/'
TEST_PATH = '../input/tsma-202223-music-genre-classification/test/Test/'
test_files_csv = pd.read_csv("../input/tsma-202223-music-genre-classification/test/.csv" , header=0)
test = pd.read_csv('../input/tsma-202223-music-genre-classification/test.csv', header=0)
train = pd.read_csv('../input/tsma-202223-music-genre-classification/train.csv', header=0)
genres = pd.read_csv('../input/tsma-202223-music-genre-classification/genres.csv', header=0)

# Classification en utilisant le Deep Learning

### Class Neural Network

In [60]:
import torch
import torchvision
import torch.nn.functional as F
from torch import binary_cross_entropy_with_logits, nn
from torch.utils.data import DataLoader, random_split
import math  

class Net(nn.Module):
    
    def __init__(self, input):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(313*128, input)
        self.fc2 = nn.Linear(input, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 8)
        
    def forward(self, x):
        x = x.view(-1, 313*128)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

### melspecro importé utilisé pour entrainer le NN

In [6]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
       os.remove(os.path.join(dirname, filename))

In [19]:
#!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_train_new.pickle
#!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_genres_train_new.pickle

#!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_songs_test_new.pickle
#!wget http://dept-info.labri.fr/~hanna/ProjetClassif/melspectro_filenames_test.pickle

import pickle

song_train=pickle.load(open('melspectro_songs_train_new.pickle','rb'))
genres_train=pickle.load(open('melspectro_genres_train_new.pickle','rb'))

print(song_train.shape, genres_train.shape)

TRACK = 2 # par exemple

#one song
print(song_train[TRACK].shape)
# genre : one hot encoding
print(genres_train[TRACK])


_songs=pickle.load(open('melspectro_songs_test_new.pickle','rb'))
_track_id=pickle.load(open('melspectro_filenames_test.pickle','rb'))

print(_songs.shape, _track_id.shape)

TRACK = 2 # par exemple

#one song
print(_songs[TRACK].shape)
# genre : one hot encoding
print(_track_id[TRACK])


(3995, 313, 128) (3995, 8)

(313, 128)

[0. 0. 1. 0. 0. 0. 0. 0.]

(4002, 313, 128) (4002,)

(313, 128)

000190


### Préparation des données

In [20]:
from torch.utils.data import Dataset, TensorDataset
from torch import Tensor

class CustomDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,x_train, y_train, transform=None):
        """
        Args:
            x_train (numpy array): data to train
            y_train (numpy array): labels
        """
        self.x = x_train
        self.y = y_train
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        label = self.y[idx]
        x_data = self.x[idx]

        if self.transform:
            x_data = self.transform(x_data)
            label = self.transform(label)

        return x_data, label

In [21]:
from sklearn.model_selection import train_test_split
from torchvision import transforms

batch_size = 64
valid_size = 0.2
x_train, x_val, y_train, y_val = train_test_split(song_train, genres_train, test_size=valid_size)
print(x_train.shape)
print(x_val.shape)
#x_train = x_train.reshape(-1, 1, 313, 128)
#x_val = x_val.reshape(-1,1, 313,128)
train_data = CustomDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
train_dataloader = DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

for x,y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {x.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

valid_data = CustomDataset(torch.from_numpy(x_val), torch.from_numpy(y_val))
validation_dataloader = DataLoader(valid_data, batch_size=batch_size,
                                   shuffle=True, num_workers=2)
_songs = np.asarray(_songs)
test_dataloader = DataLoader(torch.Tensor(_songs), batch_size=batch_size, shuffle=True, num_workers=2)


(3196, 313, 128)

(799, 313, 128)

Shape of X [N, C, H, W]: torch.Size([64, 313, 128])

Shape of y: torch.Size([64, 8]) torch.float32


### **Training**

In [79]:
from statistics import mean
model = Net(128)
print(model)
nb_batch = len(train_dataloader)
print("nb batch {}".format(nb_batch))
def validation(model, validation_data, loss_fn):
    model.train(False)
    model.eval()
    acc = {}
    acc["val_loss"] = []
    acc["val_acc"] = []
    for i, (x, y) in enumerate(validation_data):
        out = model(x)
        loss = loss_fn(out, y.float())
        print(out.shape)
        accur = accuracy(out, y)
        acc['val_loss'].append(loss.detach())
        acc['val_acc'].append(accur.double())
    epoch_loss = torch.stack([x for x in acc["val_loss"]]).mean()
    epoch_acc = torch.stack([x for x in acc["val_acc"]]).mean()
    return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}

def accuracy(prediction, y):
    pred_y = torch.max(prediction, 1)[1].data.squeeze()
    pred_y = F.one_hot(pred_y, num_classes=8)
    accuracy = (pred_y == y)
    return accuracy

def training(loss_fn, optimizer, epochs, model, train, validation_data):
    acc_arr = []
    loss_arr = []
    valid_arr = []
    model.train(True)
    model.double()
    for e in range(epochs):
        for i, (x, y) in enumerate(train):
            optimizer.zero_grad()
            prediction = model(x)
            loss = loss_fn(prediction, y.float())
            loss_arr.append(loss)
            loss.backward()
            optimizer.step()
            
            acc = accuracy(prediction, y)
            acc_arr.append(acc)
                
        valid = validation(model, validation_data, loss_fn)
        valid_arr.append(valid)
        print("Epoch: {} \t loss: {:.6f} \t acc: {:.3f}".format(e+1, loss.data.item(), torch.mean(torch.Tensor(acc_arr))))
        print("\t\t val loss: {:.3f}% \t val acc: \t {:.3f}%".format(valid["val_loss"], valid["val_acc"]))
    return acc_arr, loss_arr, valid_arr

def predict(model, test_data):
    pred = []
    model.train(False)
    model.eval()
    model.double()
    for i, x in enumerate(test_data):
        prediction = model(x.double())
        pred_np = prediction.cpu().detach().numpy()
        for n in pred_np:
            pred.append(n)
    return pred

Net(

  (fc1): Linear(in_features=40064, out_features=128, bias=True)

  (fc2): Linear(in_features=128, out_features=64, bias=True)

  (fc3): Linear(in_features=64, out_features=32, bias=True)

  (fc4): Linear(in_features=32, out_features=8, bias=True)

)

nb batch 50


In [80]:
loss_fn = nn.CrossEntropyLoss()
lr = 1e-3
epochs=50
optimizer = torch.optim.Adam(model.parameters(), lr)
acc, loss, valid = training(loss_fn, optimizer, epochs, model, train_dataloader, validation_dataloader)

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([64, 8])

torch.Size([31, 8])


RuntimeError: stack expects each tensor to be equal size, but got [64, 8] at entry 0 and [31, 8] at entry 12

In [ ]:
res = predict(model, test_dataloader)

In [ ]:
print(sum(res[0]))
print("prediction of class for first song")
print(np.argmax(sum(res[0]))+1)

In [ ]:
def process_data(output_prediction):
    predictions = [np.argmax(sum(i))+1 for i in output_prediction]
    return predictions

prediction = process_data(res)

In [ ]:
# valeurs manquantes dans le .csv
tab = ['059684', '098565', '098568', '098569', '098571', '098559']
idx=0
test["genre_id"] = 1
for index, row in test.iterrows():
    track_id = f'{row.track_id:06d}'
    if track_id in tab:
        row["genre_id"] = 1
    else:
        row["genre_id"] = prediction[idx]
        idx+=1

test.to_csv("submission.csv", encoding='utf-8', index=False)

In [98]:
print(test)

      track_id  genre_id

0           10         6

1          141         6

2          190         6

3          193         6

4          194         6

...        ...       ...

4003    155065         6

4004    155066         6

4005    155141         6

4006    155298         6

4007    155306         6



[4008 rows x 2 columns]
